In [170]:
%load_ext autoreload
%autoreload 2
import psycopg2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
from pandas import DataFrame, read_sql
#import seaborn as sns
#sns.set_style("whitegrid")
from pandas import DataFrame, to_datetime
from IPython.display import display
path = os.path.expanduser('~')
import datetime as dt
import pandas.io.sql as psql

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Conexion a PostgreSQL y para obtener DF con la data

In [171]:
conn = ""
try:
        conn = psycopg2.connect(dbname='dw_blocketdb_ch', user='bnbiuser', host='200.29.173.148', password='VE1bi@BN112AzLkOP')
except:
        logger.info('ERROR: I am unable to connect to the database')
        print
        "I am unable to connect to the database"
        
query = """
DROP TABLE stg.temp_visits_content;
CREATE TABLE 
stg.temp_visits_content as
select
x.date_start::date,
x.list_id::int,
x.visits::int
from
stg.fact_weekly_adview_xiti x
where
date_start::date in
(
'2018-09-03',
'2018-09-10',
'2018-09-17',
'2018-09-24',
'2018-10-01',
'2018-10-08',
'2018-10-15',
'2018-10-22',
'2018-10-29'
)
and list_id <> '-';

select
a.ad_id_pk,
a.approval_date::date,
case when ad_type_id_fk = 1 then 'sell' else 'buy' end as ad_type,
cm.category_main_id_pk,
cm.category_main_name,
a.category_id_fk,
c.category_name,
pl.platform_name,
r.region_name,
case when sp.seller_id_fk is not null then 'pro' else 'pri' end as pri_pro_seller,
a.subject,
a.body,
a.price,
case when im.ad_id_nk is null then 'no' else 'yes' end as has_a_photo,
im.photos,
im.mean_resol,
p.params,
x.visits
from ods.ad a inner join
ods.category c on c.category_id_pk = a.category_id_fk
left join
ods.category_main cm on cm.category_main_id_pk = c.category_main_id_fk
left join
ods.platform pl on pl.platform_id_pk = a.platform_id_fk
left join
ods.region r on r.region_id_pk = a.region_id_fk
left join
stg.temp_images_per_ad im on im.ad_id_nk = a.ad_id_nk
left join
stg.temp_params_content_agg p on p.ad_id = a.ad_id_nk 
left join
ods.seller_pro_details sp on sp.seller_id_fk = a.seller_id_fk and sp.category_id_fk = a.category_id_fk
inner join
stg.temp_visits_content x on x.date_start::date = a.approval_date::date and x.list_id::int = a.list_id_nk::int
where a.approval_date::date between '2018-09-01' and '2018-10-31'
and ad_type_id_fk = 1
and pl.platform_id_pk in (1,2,5,6,7,8);
"""
df_sql = psql.read_sql(query, conn)
#conn.close()
#return df_sql

In [172]:
df_sql.shape

(301674, 18)

In [173]:
df_guradar = df_sql

In [195]:
df=df_guradar.copy()

In [196]:
df=df[(df.category_id_fk == 30)].copy()

In [197]:
df.shape

(28645, 18)

### agregar caracteristicas de la descripcion al DF

In [198]:
df['len_subject'] = df['subject'].str.split(" ").apply(len)
df['len_body'] = df['body'].str.split(" ").apply(len)
df['len_full_description'] = df['len_subject'] + df['len_body']

In [199]:
df


,ad_id_pk,approval_date,ad_type,category_main_id_pk,category_main_name,category_id_fk,category_name,platform_name,region_name,pri_pro_seller,...,body,price,has_a_photo,photos,mean_resol,params,visits,len_subject,len_body,len_full_description
1367,71509567,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,Regi�n Metropolitana,pri,...,Totalmente funcinal. Con un detalle en la esqu...,20000.0,yes,3.0,217.0,1.0,73,2,20,22
1370,71506728,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,IV Coquimbo,pri,...,El equipo se encuentra en perfecto estado incl...,75000.0,yes,3.0,196.0,1.0,59,5,21,26
1373,71502709,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,Regi�n Metropolitana,pri,...,Equipo Motorola x play\n16/2\n21/5\nTrizado ab...,40000.0,yes,5.0,253.0,1.0,48,3,7,10
1374,71509832,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,X Los Lagos,pri,...,"Vendo iPhone 6 de 32gb en muy buen estado, sie...",150000.0,yes,5.0,172.0,1.0,47,5,57,62
1375,71510522,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",M Site,II Antofagasta,pri,...,Iphone 6 plus de 16gb impeke cero detalles lib...,210000.0,yes,4.0,0.0,1.0,44,3,31,34
1378,71485460,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",M Site,IV Coquimbo,pri,...,"Vendo celular J7 Pro dorado dual SIM, con todo...",120000.0,yes,6.0,0.0,NaN,43,3,43,46
1379,71487663,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,Regi�n Metropolitana,pri,...,Solo para claro\nPantalla en negro\n se vende ...,10000.0,yes,2.0,129.0,1.0,42,2,16,18
1381,71477164,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",Web,Regi�n Metropolitana,pri,...,"Vendo Iphone en buen estado, est�ticamente sin...",40000.0,yes,6.0,0.0,1.0,38,4,14,18
1382,71490167,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",Web,I Tarapac�,pri,...,Vendo celular con pantalla reci�n cambiada per...,30000.0,yes,2.0,0.0,1.0,38,3,19,22
1386,71495536,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,IV Coquimbo,pri,...,Vendo celular Huawei modelo ATU-LX3 en perfect...,80000.0,yes,3.0,172.0,1.0,37,3,16,19


### parametros para definir tipo de AD (precio, parametros)

In [200]:
# Mustbe price
df['must_be_price'] = ((df['ad_type'] == 'sell') | (df['ad_type'] == 'buy')).astype(int)

In [201]:
# Price missing
df.loc[(df['must_be_price'] == 1) & (df['price'].isnull()),'price_missing'] = 1
df.loc[(df['must_be_price'] == 1) & (-df['price'].isnull()),'price_missing'] = 0

In [202]:
df['price_log'] = np.log(df.price + 1)
df['price_normalized'] = (df.price-np.min(df.price))/(np.max(df.price)-np.min(df.price))

In [203]:
df

,ad_id_pk,approval_date,ad_type,category_main_id_pk,category_main_name,category_id_fk,category_name,platform_name,region_name,pri_pro_seller,...,mean_resol,params,visits,len_subject,len_body,len_full_description,must_be_price,price_missing,price_log,price_normalized
1367,71509567,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,Regi�n Metropolitana,pri,...,217.0,1.0,73,2,20,22,1,0.0,9.903538,1.619919e-04
1370,71506728,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,IV Coquimbo,pri,...,196.0,1.0,59,5,21,26,1,0.0,11.225257,6.074919e-04
1373,71502709,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,Regi�n Metropolitana,pri,...,253.0,1.0,48,3,7,10,1,0.0,10.596660,3.239919e-04
1374,71509832,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,X Los Lagos,pri,...,172.0,1.0,47,5,57,62,1,0.0,11.918397,1.214992e-03
1375,71510522,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",M Site,II Antofagasta,pri,...,0.0,1.0,44,3,31,34,1,0.0,12.254868,1.700992e-03
1378,71485460,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",M Site,IV Coquimbo,pri,...,0.0,NaN,43,3,43,46,1,0.0,11.695255,9.719919e-04
1379,71487663,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,Regi�n Metropolitana,pri,...,129.0,1.0,42,2,16,18,1,0.0,9.210440,8.099190e-05
1381,71477164,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",Web,Regi�n Metropolitana,pri,...,0.0,1.0,38,4,14,18,1,0.0,10.596660,3.239919e-04
1382,71490167,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",Web,I Tarapac�,pri,...,0.0,1.0,38,3,19,22,1,0.0,10.308986,2.429919e-04
1386,71495536,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,IV Coquimbo,pri,...,172.0,1.0,37,3,16,19,1,0.0,11.289794,6.479919e-04


In [204]:
params_cat_no_mandatory = pd.read_csv('/Users/Javier/Jupyter/params_per_category_no_mandatory.csv',sep=';')
params_cat_no_mandatory['must_have_params'] = 1
params_cat_no_mandatory

,category_id_fk,site_category_name,total_params_per_category,must_have_params
0,7,"Autos, camionetas y 4x4",1,1
1,8,Camiones y furgones,1,1
2,9,Motos,1,1
3,30,"Celulares, teléfonos y accesorios",1,1
4,32,Ofertas de empleo,2,1
5,37,Moda y vestuario,2,1
6,38,"Bolsos, bisutería y accesorios",1,1
7,47,Comprar,6,1
8,48,Arrendar,6,1
9,49,Arriendo de temporada,2,1


In [205]:
df = df.merge(params_cat_no_mandatory[['category_id_fk', 'total_params_per_category','must_have_params']],how='left',on = 'category_id_fk')

In [206]:
df['total_params_per_category'] = df['total_params_per_category'].fillna(0)
df['must_have_params'] = df['must_have_params'].fillna(0)

In [207]:
df

,ad_id_pk,approval_date,ad_type,category_main_id_pk,category_main_name,category_id_fk,category_name,platform_name,region_name,pri_pro_seller,...,visits,len_subject,len_body,len_full_description,must_be_price,price_missing,price_log,price_normalized,total_params_per_category,must_have_params
0,71509567,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,Regi�n Metropolitana,pri,...,73,2,20,22,1,0.0,9.903538,1.619919e-04,1,1
1,71506728,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,IV Coquimbo,pri,...,59,5,21,26,1,0.0,11.225257,6.074919e-04,1,1
2,71502709,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,Regi�n Metropolitana,pri,...,48,3,7,10,1,0.0,10.596660,3.239919e-04,1,1
3,71509832,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,X Los Lagos,pri,...,47,5,57,62,1,0.0,11.918397,1.214992e-03,1,1
4,71510522,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",M Site,II Antofagasta,pri,...,44,3,31,34,1,0.0,12.254868,1.700992e-03,1,1
5,71485460,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",M Site,IV Coquimbo,pri,...,43,3,43,46,1,0.0,11.695255,9.719919e-04,1,1
6,71487663,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,Regi�n Metropolitana,pri,...,42,2,16,18,1,0.0,9.210440,8.099190e-05,1,1
7,71477164,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",Web,Regi�n Metropolitana,pri,...,38,4,14,18,1,0.0,10.596660,3.239919e-04,1,1
8,71490167,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",Web,I Tarapac�,pri,...,38,3,19,22,1,0.0,10.308986,2.429919e-04,1,1
9,71495536,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,IV Coquimbo,pri,...,37,3,16,19,1,0.0,11.289794,6.479919e-04,1,1


In [208]:
params_per_cat = params_cat_no_mandatory.groupby('site_category_name').agg({'total_params_per_category':'sum'})
params_per_cat['site_category_name'] = params_per_cat.index
params_per_cat.columns = ['total_params_per_category', 'site_category_name']
params_per_cat.head(100)

,total_params_per_category,site_category_name
site_category_name,,
Arrendar,6,Arrendar
Arriendo de temporada,2,Arriendo de temporada
"Autos, camionetas y 4x4",1,"Autos, camionetas y 4x4"
"Bolsos, bisutería y accesorios",1,"Bolsos, bisutería y accesorios"
Calzado,1,Calzado
Camiones y furgones,1,Camiones y furgones
"Celulares, teléfonos y accesorios",1,"Celulares, teléfonos y accesorios"
Comprar,6,Comprar
Moda y vestuario,2,Moda y vestuario


## adding ad_reply first week

In [232]:
conn = ""
try:
        conn = psycopg2.connect(dbname='dw_blocketdb_ch', user='bnbiuser', host='200.29.173.148', password='VE1bi@BN112AzLkOP')
except:
        logger.info('ERROR: I am unable to connect to the database')
        print
        "I am unable to connect to the database"
        
query = """
select
t.ad_id_pk,
count(t.ad_reply_id_pk) as ad_reply_first_week
from 
(
select 
a.ad_id_pk,
r.ad_reply_id_pk,
case when (r.ad_reply_creation_date::date - a.approval_date::date)::int <= 7 then 1 else 0 end as ad_reply_first_week
from
ods.ad_reply r 
inner join
ods.ad a on a.ad_id_pk = r.ad_id_fk and a.approval_date::date >= '2018-09-01'
where
r.ad_reply_creation_date::date >= '2018-09-01'
)t
where 
ad_reply_first_week = 1
group by 1
"""
df_ar = psql.read_sql(query, conn)



In [234]:
table_ad_reply = df_ar

## adding product name

In [242]:
conn = ""
try:
        conn = psycopg2.connect(dbname='dw_blocketdb_ch', user='bnbiuser', host='200.29.173.148', password='VE1bi@BN112AzLkOP')
except:
        logger.info('ERROR: I am unable to connect to the database')
        print
        "I am unable to connect to the database"
        
query = """
select 
ad_id_fk as ad_id_pk,
product as product_name
from
stg.ad_product
where 
approval_date::date >= '2018-09-01'
and product not like 'None%'
"""
df_prod = psql.read_sql(query, conn)



In [243]:
table_prod_name = df_prod

### Adding information about the quartiles for price, length full_description, number of photos, % params per Celulares, telefonos y accesorios

In [209]:
# Function to compute extremal values and quartiles
def q010(x):
    return x.quantile(0.10)

def q1(x):
    return x.quantile(0.25)

def q2(x):
    return x.quantile(0.50)

def q3(x):
    return x.quantile(0.75)

def q090(x):
    return x.quantile(0.90)

f = [q010,q1,q2,q3,q090]

In [210]:
table_len_full_description = df.groupby(['category_name']).agg({'len_full_description':f}).reset_index()
table_len_full_description.head(100)

category_name len_full_description                 
                                                     q010  q1  q2  q3 q090
0  Celulares, tel�fonos y accesorios                   10  15  25  40   66

In [211]:
table_price = df[(-df['price'].isnull()) & (df.must_be_price==1)].groupby(['category_name']).agg({'price':f}).reset_index()
table_price.head(100)

category_name    price                              \
                                         q010       q1       q2        q3   
0  Celulares, tel�fonos y accesorios  10000.0  35000.0  75000.0  150000.0   

             
       q090  
0  250000.0

In [212]:
table_nof_photos = df.groupby(['category_name']).agg({'photos':f}).reset_index()
table_nof_photos.head(100)

category_name photos                    
                                       q010   q1   q2   q3 q090
0  Celulares, tel�fonos y accesorios    1.0  2.0  3.0  5.0  6.0

In [213]:
table_completness_in_params = df.groupby(['category_name']).agg({'params':f}).reset_index()
table_completness_in_params.head(100)

category_name params                    
                                       q010   q1   q2   q3 q090
0  Celulares, tel�fonos y accesorios    1.0  1.0  1.0  1.0  1.0

### Apply percentiles


In [214]:
categories = df.category_name.unique()

In [215]:
huge = 100000000000000000
labels = ['q010','q1','q2','q3','q090', 'q100']
labels_num = [0.10,0.33,0.5,0.66,0.90,1]

In [216]:
def apply_percentiles(cat,table,feature,huge):
    # where we have to change the values
    where_quart = (table['category_name'] == cat)
    where_df = df['category_name'] == cat
    # extracting the values of quartile per this cat and reg
    if np.sum(where_quart) > 0:
        aux = table[where_quart][feature].reset_index()
        table_quartiles = [0,aux['q010'][0]+0.001,aux['q1'][0]+0.002,aux['q2'][0]+0.003,aux['q3'][0]+0.004,aux['q090'][0]+0.005,huge]
        # apply to the new column
        if np.sum(where_df) > 0:
            df.loc[where_df, feature+'_quartil'] = pd.cut(df[where_df][feature],
                                                          bins = table_quartiles,
                                                          labels = labels)
            df.loc[where_df, feature+'_quartil_num'] = pd.cut(df[where_df][feature],
                                                          bins = table_quartiles,
                                                          labels = labels_num)

In [217]:
for cat in categories:
    apply_percentiles(cat,table_completness_in_params,'params',huge)
    apply_percentiles(cat,table_len_full_description,'len_full_description',huge)
    apply_percentiles(cat,table_nof_photos,'photos',huge)
    apply_percentiles(cat,table_price,'price',huge)

In [218]:
table_price.columns = table_price.columns.droplevel() + '_price'
table_price.columns = ['category_name', 'q010_price', 'q1_price', 'q2_price', 'q3_price',
       'q090_price']

In [219]:
table_price.head(4)

,category_name,q010_price,q1_price,q2_price,q3_price,q090_price
0,"Celulares, tel�fonos y accesorios",10000.0,35000.0,75000.0,150000.0,250000.0


In [220]:
table_nof_photos.columns = table_nof_photos.columns.droplevel() + '_nof_photos'
table_nof_photos.columns = ['category_name', 'q010_nof_photos', 'q1_nof_photos',
                            'q2_nof_photos','q3_nof_photos', 'q090_nof_photos']

In [221]:
table_nof_photos.head()

,category_name,q010_nof_photos,q1_nof_photos,q2_nof_photos,q3_nof_photos,q090_nof_photos
0,"Celulares, tel�fonos y accesorios",1.0,2.0,3.0,5.0,6.0


In [222]:
table_len_full_description.columns = table_len_full_description.columns.droplevel() + '_len_full_description'
table_len_full_description.columns = ['category_name',
       'q010_len_full_description', 'q1_len_full_description',
       'q2_len_full_description', 'q3_len_full_description',
       'q090_len_full_description']

In [223]:
table_len_full_description.head()

,category_name,q010_len_full_description,q1_len_full_description,q2_len_full_description,q3_len_full_description,q090_len_full_description
0,"Celulares, tel�fonos y accesorios",10,15,25,40,66


In [224]:
table_completness_in_params.columns = table_completness_in_params.columns.droplevel() + '_completness_in_params'
table_completness_in_params.columns = ['category_name', 'q010_completness_in_params',
       'q1_completness_in_params', 'q2_completness_in_params',
       'q3_completness_in_params', 'q090_completness_in_params']

In [225]:
table_completness_in_params.head()

,category_name,q010_completness_in_params,q1_completness_in_params,q2_completness_in_params,q3_completness_in_params,q090_completness_in_params
0,"Celulares, tel�fonos y accesorios",1.0,1.0,1.0,1.0,1.0


### Agregar al DF principal

In [226]:
df  = df.merge(table_price,how='left',on = 'category_name')          

In [227]:
df= df.merge(table_nof_photos,how='left',on = 'category_name')

In [228]:
df = df.merge(table_completness_in_params,how='left',on = 'category_name')

In [229]:
df = df.merge(table_len_full_description,how='left',on = 'category_name')

In [239]:
df = df.merge(table_ad_reply,how='left',on = 'ad_id_pk')
#df = df.drop(columns='ad_reply_first_week_y')

In [244]:
df = df.merge(table_prod_name,how='left',on = 'ad_id_pk')

In [245]:
df

,ad_id_pk,approval_date,ad_type,category_main_id_pk,category_main_name,category_id_fk,category_name,platform_name,region_name,pri_pro_seller,...,q2_completness_in_params,q3_completness_in_params,q090_completness_in_params,q010_len_full_description,q1_len_full_description,q2_len_full_description,q3_len_full_description,q090_len_full_description,ad_reply_first_week,product_name
0,71509567,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,Regi�n Metropolitana,pri,...,1.0,1.0,1.0,10,15,25,40,66,9.0,huawei
1,71506728,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,IV Coquimbo,pri,...,1.0,1.0,1.0,10,15,25,40,66,4.0,iphone 5s
2,71502709,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,Regi�n Metropolitana,pri,...,1.0,1.0,1.0,10,15,25,40,66,1.0,motorola x play
3,71509832,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,X Los Lagos,pri,...,1.0,1.0,1.0,10,15,25,40,66,4.0,iphone 6
4,71510522,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",M Site,II Antofagasta,pri,...,1.0,1.0,1.0,10,15,25,40,66,5.0,iphone 6 plus
5,71485460,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",M Site,IV Coquimbo,pri,...,1.0,1.0,1.0,10,15,25,40,66,1.0,samsung galaxy j7
6,71487663,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,Regi�n Metropolitana,pri,...,1.0,1.0,1.0,10,15,25,40,66,3.0,samsung galaxy j5
7,71477164,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",Web,Regi�n Metropolitana,pri,...,1.0,1.0,1.0,10,15,25,40,66,7.0,iphone 5c
8,71490167,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",Web,I Tarapac�,pri,...,1.0,1.0,1.0,10,15,25,40,66,2.0,Otros Computadores & electr�nica
9,71495536,2018-10-01,sell,3,Computadores & electr�nica,30,"Celulares, tel�fonos y accesorios",NGA Android,IV Coquimbo,pri,...,1.0,1.0,1.0,10,15,25,40,66,NaN,huawei


In [230]:
with pd.option_context('display.max_columns', None):
    print(df)

       ad_id_pk approval_date ad_type  category_main_id_pk  \
0      71509567    2018-10-01    sell                    3   
1      71506728    2018-10-01    sell                    3   
2      71502709    2018-10-01    sell                    3   
3      71509832    2018-10-01    sell                    3   
4      71510522    2018-10-01    sell                    3   
5      71485460    2018-10-01    sell                    3   
6      71487663    2018-10-01    sell                    3   
7      71477164    2018-10-01    sell                    3   
8      71490167    2018-10-01    sell                    3   
9      71495536    2018-10-01    sell                    3   
10     71478771    2018-10-01    sell                    3   
11     71495344    2018-10-01    sell                    3   
12     71478031    2018-10-01    sell                    3   
13     71500326    2018-10-01    sell                    3   
14     71475408    2018-10-01    sell                    3   
15     7

### crear dataset en csv

In [246]:
df.to_csv('/Users/Javier/Jupyter/dataset_for_ad_index_phone_accurated_params_no_mandatory.csv', index= False)
#, sep=';')